In [ ]:
import rioxarray
import geopandas as gpd

import pystac_client
import odc.stac

import folium
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from rasterio import features
import rasterio

from xrspatial import zonal_stats

In [ ]:
data_path = "assignment_data"

In [ ]:
output_folder = "output_group2"

# Create clip dataset Rockhampton

# Copernicus API Data Download

## Create geom and bbox

In [ ]:
def subtract_area_geom(path, name):
    """
    Function to load shapefile, reproject and return geometry and bbox
    """
    #load geodataframe
    gdf_area = gpd.read_file(path)

    #Filter relevant subset
    rockhampton = gdf_area[gdf_area['NAME_2'] == name]

    #reproject data
    rockhampton_reproj = rockhampton.to_crs(32755)

    #store geometry and bbox into variables
    bbox = rockhampton.total_bounds
    geom = rockhampton_reproj['geometry']

    return geom, bbox, rockhampton_reproj #, rockhampton
    

In [ ]:
geom, bbox, rockhampton_reproj = subtract_area_geom(f'{data_path}/study_area/ADM_ADM_2.shp', 'Rockhampton')

## Satellite data

In [ ]:
def calculate_ndwi(bbox, geom):

    """
    Function to: 
    
    - access the sentinel2 api. Select timeframe and location and download matching tiles
    into a data cube.
    - Filter the returned data to the previdously created bbox
    - subtrackt the relevant bands
    - mask clouds
    - calculate NDWI
    
    """
    #connect to api
    api_url = "https://earth-search.aws.element84.com/v1"
    collection_id = "sentinel-2-l2a"
    
    client = pystac_client.Client.open(api_url)
    search = client.search(
        collections=[collection_id],
        datetime="2017-04-01/2017-08-01",
        bbox= bbox
       # query=['eo:cloud_cover<3']
    )
    
    item_collection = search.item_collection()

    #select loading criteria
    ds = odc.stac.load(
        item_collection,
        groupby='solar_day',
        chunks={'x': 2048, 'y': 2048},
        use_overviews=True,
        resolution=80,
        bbox= bbox#reference_df.total_bounds,
    )

    #select bands
    red = ds['red']
    blue = ds['blue']
    green = ds['green']
    nir = ds['nir']
    scl = ds['scl']
    #visual = ds['visual']

    #clip bands to geometry

    blue_clip = blue.rio.clip(geom)
    red_clip = red.rio.clip(geom)
    nir_clip = nir.rio.clip(geom)
    green_clip = green.rio.clip(geom)

    #MASK REMOVES CRS

    # generate mask ("True" for pixel being cloud or water)
    mask = scl.isin([
        3,  # CLOUD_SHADOWS
        8,  # CLOUD_MEDIUM_PROBABILITY
        9,  # CLOUD_HIGH_PROBABILITY
        10  # THIN_CIRRUS
    ])
    nir_masked = nir_clip.where(~mask)
    green_masked = green_clip.where(~mask)
    print('check')


    #calculate ndwi
    ndwi = (green_masked - nir_masked) / (green_masked + nir_masked)

    return ndwi, red_clip, green_clip, blue_clip

In [ ]:
ndwi,red_clip, green_clip, blue_clip = calculate_ndwi(bbox, geom)

In [ ]:
ndwi.sel(time= "2017-04-08").plot()

In [ ]:
def subset_ndwi_on_date(ndwi,date, output_folder):

    """
    Function to subtract ndwi for specific date out of data cube, than mask flooded area 
    and save as a file
    
    
    """
    ndwi_date = ndwi.sel(time= date)

    flooded_area = (
        (ndwi_date > -0.3)
        )

    filename_area = output_folder +'/flooded2_' + str(date) + '.tif'

    flooded_area.rio.to_raster(filename_area, dtype='int8')

    return filename_area

In [ ]:
dates = ["2017-04-08","2017-04-28","2017-07-27"]

In [ ]:
ndwi_paths = {}
for date in dates:
    print(date)
    file = subset_ndwi_on_date(ndwi,date, output_folder)
    print(file)
    opened_file = rioxarray.open_rasterio(file)
    ndwi_paths[date]= opened_file

In [ ]:
ndwi_paths['2017-04-08'].plot()

# Rasterize OSM data

#### Load OSM data

In [ ]:
osm_lines = gpd.read_file(f'{data_path}/osm_data/osm_lines_new.gpkg')

#### explore the data

In [ ]:
#osm_lines.explore()

In [ ]:
#osm_lines.columns

In [ ]:
#osm_lines.head()

In [ ]:
#unique types of roads
#osm_lines['highway'].unique()

In [ ]:
# unique man made lines
#osm_lines['man_made'].unique()

#### reproject the osm data to later rasterize it

In [ ]:
#reproject osm data 
osm_lines = osm_lines.to_crs(rockhampton_reproj.crs)

#### use sjoin to select the osm data that is within the rockhampton municipality

In [ ]:
# filter osm data to rockhampton
rockhampton_lines = gpd.sjoin(osm_lines,rockhampton_reproj, how='inner')

In [ ]:
#rockhampton_lines.explore()

#### function to 
##### - filter the data
##### - create a buffer
##### - add a 'code' column to the dataset

In [ ]:
def filter_and_buffer_osm_data(filter_column: str, filter_attributs: list, code: int, buffer_dist: int ):
    """
    function to 

    - filter the data
    - create a buffer
    - add a 'code' column to the dataset
    """
    attribute_list = filter_attributs
    filtered_data = rockhampton_lines[rockhampton_lines[filter_column].isin(attribute_list)][['geometry']]
    filtered_data.columns = ['geometry']
    filtered_data['code'] = code
    filtered_data_buffer = filtered_data.buffer(buffer_dist)
    data = {"geometry":filtered_data_buffer , "type": "Main Roads", "code": code}
    assets_area = gpd.GeoDataFrame(data)
    return assets_area
    

In [ ]:
roads_buffer = 50
power_buffer = 100

In [ ]:
main_roads_filter = ['primary', 'secondary', 'tertiary']
main_roads_area = filter_and_buffer_osm_data('highway', main_roads_filter, 1, roads_buffer)

In [ ]:
power_filter = ['power']
power_area = filter_and_buffer_osm_data('man_made', power_filter, 2, power_buffer)

#### merge datasets

In [ ]:
#concat powerlines and roads
assets_area = pd.concat([main_roads_area,power_area]).reset_index(drop=True)

In [ ]:
assets_area.plot(column='code',legend=True,cmap='Set1')

#### Rasterize Assets data

In [ ]:
assets_geom = assets_area[["geometry", "code"]].values.tolist()

In [ ]:
flooded_sqeezed = ndwi_paths[dates[0]].squeeze()

In [ ]:
assets_rasterized = features.rasterize(assets_geom, out_shape = flooded_sqeezed.shape, transform = ndwi_paths["2017-04-08"].rio.transform())

In [ ]:
assets_rasterized_xarr = flooded_sqeezed.copy()
assets_rasterized_xarr.data = assets_rasterized
assets_rasterized_xarr.plot()

# buildings
Buildings from 2018 were chosen as indicator for the damage to the population, because it indicates urban areas wehere a lot of people live and can be affected by the floods. We dis not use the population dataset because it had a 100 meter and it would influence the results of the affected area. Although the buildings are from 2018 and therefore there is a possibility of buildings already being destroyed by the floods, it is considered a better indication due to its 10 metre resolution.

In [ ]:
buidling_path = "building_data/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C33.tif"
buildings_2015 = rioxarray.open_rasterio(buidling_path, overview_level = 2)

In [ ]:
clip_layer = rockhampton_reproj.to_crs(buildings_2015.rio.crs)

In [ ]:
bbox = clip_layer.total_bounds

In [ ]:
buildings_clipped = buildings_2015.rio.clip_box(*bbox)

#### change coordinate system of raster and resolution

#### legend:
##### 01 : MSZ, open spaces, low vegetation surfaces NDVI <= 0.3
##### 02 : MSZ, open spaces, medium vegetation surfaces 0.3 < NDVI <=0.5
##### 03 : MSZ, open spaces, high vegetation surfaces NDVI > 0.5
##### 04 : MSZ, open spaces, water surfaces LAND < 0.5
##### 05 : MSZ, open spaces, road surfaces
##### 11 : MSZ, built spaces, residential, building height <= 3m
##### 12 : MSZ, built spaces, residential, 3m < building height <= 6m
##### 13 : MSZ, built spaces, residential, 6m < building height <= 15m
##### 14 : MSZ, built spaces, residential, 15m < building height <= 30m
##### 15 : MSZ, built spaces, residential, building height > 30m
##### 21 : MSZ, built spaces, non-residential, building height <= 3m
##### 22 : MSZ, built spaces, non-residential, 3m < building height <= 6m
##### 23 : MSZ, built spaces, non-residential, 6m < building height <= 15m
##### 24 : MSZ, built spaces, non-residential, 15m < building height <= 30m
##### 25 : MSZ, built spaces, non-residential, building height > 30m
#### NoData [255]

In [ ]:
buildings_clipped_filtered = (
    (buildings_clipped>= 11)
)

In [ ]:
buildings_clipped_filtered.plot()

In [ ]:
buildings_clipped_filtered_squeezed = buildings_clipped_filtered.squeeze()

In [ ]:
buildings_clipped_filtered_squeezed.rio.to_raster("buildings_clipped_filtered.tif", dtype = "int8")

In [ ]:
buildings_clipped_filtered_raster = rioxarray.open_rasterio("buildings_clipped_filtered.tif")

In [ ]:
match_layer = red_clip

In [ ]:
buildings_clipped_filtered_match = buildings_clipped_filtered_raster.rio.reproject_match(match_layer)

# Zonal Statistics

In [ ]:
# for date, raster in ndwi_paths.items():
#     print(date)
#     print(raster)

In [ ]:

def apply_zonal_statistics(satteltite_paths: dict, assets, label: str ):
    zonal_statistics_dict = {}
    assets_squeeze = assets.squeeze()
    for date, raster in ndwi_paths.items():
        raster_squeeze = raster.squeeze()
        zonal_statistics = zonal_stats(assets_squeeze, raster_squeeze )
        zonal_statistics['sum_ha'] = zonal_statistics['sum'] * 80 / 10_000
        zonal_statistics_dict[f"{label}_{date}"] = zonal_statistics
        print(date)
        print(zonal_statistics)
    return zonal_statistics_dict
        
    
    

In [ ]:
stats_assets_infrastructure = apply_zonal_statistics(ndwi_paths, assets_rasterized_xarr, "assets_infrastructure")

In [ ]:
stats_assets_building = apply_zonal_statistics(ndwi_paths, buildings_clipped_filtered_match, "assets_building")

## Subtracks zonal stats

In [ ]:
flood_building_0804 = stats_assets_building['assets_building_2017-04-08'].iloc[1]['sum_ha']
flood_building_2804 = stats_assets_building['assets_building_2017-04-28'].iloc[1]['sum_ha']
flood_building_2707 = stats_assets_building['assets_building_2017-07-27'].iloc[1]['sum_ha']

In [ ]:
flood_roads_0804 = stats_assets_infrastructure['assets_infrastructure_2017-04-08'].iloc[1]['sum_ha']
flood_roads_2804 = stats_assets_infrastructure['assets_infrastructure_2017-04-28'].iloc[1]['sum_ha']
flood_roads_2707 = stats_assets_infrastructure['assets_infrastructure_2017-07-27'].iloc[1]['sum_ha']

In [ ]:
stats_assets_infrastructure['assets_infrastructure_2017-04-08'].iloc[2]['sum_ha']

In [ ]:
flood_power_0804 = stats_assets_infrastructure['assets_infrastructure_2017-04-08'].iloc[2]['sum_ha']
flood_power_2804 = stats_assets_infrastructure['assets_infrastructure_2017-04-28'].iloc[2]['sum_ha']
flood_power_2707 = stats_assets_infrastructure['assets_infrastructure_2017-07-27'].iloc[2]['sum_ha']

In [ ]:
flood_0804 = np.sum(stats_assets_building['assets_building_2017-04-08']['sum_ha'])
flood_2804 = np.sum(stats_assets_building['assets_building_2017-04-28']['sum_ha'])
flood_2707 = np.sum(stats_assets_building['assets_building_2017-07-27']['sum_ha'])

In [ ]:
total_flood = [flood_0804,flood_2804,flood_2707]

In [ ]:
flooded_buildings = [flood_building_0804,flood_building_2804,flood_building_2707]
flooded_roads = [flood_roads_0804, flood_roads_2804, flood_roads_2707]
flooded_power = [flood_power_0804, flood_power_2804, flood_power_2707]

## Making the graph

In [ ]:
series = pd.Series(flooded_buildings)
series2 = pd.Series(total_flood)
series3 = pd.Series(flooded_roads)
series4 = pd.Series(flooded_power)
# Create a plot for the first series
fig, ax1 = plt.subplots()

ax1.plot(series, color='b', label='Flooded Buildings (ha)')
ax1.plot(series3, color='g', label='Flooded Roads (ha)')
ax1.plot(series4, color='m', label='Flooded Power (ha)')
ax1.set_xlabel('Date')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_ylim(0, 15)  # Set the y-axis range from 0 to 1500

# Define the positions and labels for x-ticks
positions = range(len(flooded_buildings))  # [0, 1, 2]
labels = ['08-04', '28-04', '27-07']

ax1.set_xticks(positions)
ax1.set_xticklabels(labels)

# Create a twin axis for the second series
ax2 = ax1.twinx()
ax2.plot(series2, color='r', label='Total Flood (ha)')
ax2.tick_params(axis='y', labelcolor='r')

# Combine legends from both axes
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

# Show the plot
fig.tight_layout()
plt.show()